<div class="alert alert-block alert-danger">

# FIT5196 Task 2 in Assessment 1
    
#### Student Name: xxxxxxx
#### Student ID: xxxxxxxxx

Date: xxxxxxxx

Environment: xxxxxx

Libraries used:
* os (for interacting with the operating system, included in Python xxxx) 
* pandas 1.1.0 (for dataframe, installed and imported) 
* multiprocessing (for performing processes on multi cores, included in Python 3.6.9 package) 
* itertools (for performing operations on iterables)
* nltk 3.5 (Natural Language Toolkit, installed and imported)
* nltk.tokenize (for tokenization, installed and imported)
* nltk.stem (for stemming the tokens, installed and imported)

    </div>

<div class="alert alert-block alert-info">
    
## Table of Contents

</div>

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Input File](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Tokenization](#tokenize) <br>
$\;\;\;\;$[4.2. Whatever else](#whetev) <br>
$\;\;\;\;$[4.3. Genegrate numerical representation](#whetev1) <br>
[5. Writing Output Files](#write) <br>
$\;\;\;\;$[5.1. Vocabulary List](#write-vocab) <br>
$\;\;\;\;$[5.2. Sparse Matrix](#write-sparseMat) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

<div class="alert alert-block alert-success">
    
## 1.  Introduction  <a class="anchor" name="Intro"></a>

This assessment concerns textual data and the aim is to extract data, process them, and transform them into a proper format. The dataset provided is in the format of a PDF file containing ....

<div class="alert alert-block alert-success">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>

In this assessment, any python packages is permitted to be used. The following packages were used to accomplish the related tasks:

* **os:** to interact with the operating system, e.g. navigate through folders to read files
* **re:** to define and use regular expressions
* **pandas:** to work with dataframes
* **multiprocessing:** to perform processes on multi cores for fast performance 
* ...

In [37]:
import os
import re
import langid
import pandas as pd
import multiprocessing
from itertools import chain
import nltk
from nltk.probability import *
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.stem import PorterStemmer
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer

-------------------------------------

<div class="alert alert-block alert-success">
    
## 3.  Examining Input File <a class="anchor" name="examine"></a>

In [38]:
df = pd.read_json("task1_30.json")
df = df.T.reset_index() 
df.columns = ['gmap_id', 'reviews', 'earliest', 'latest'] 
print(df.head(20))
for index, row in df.iterrows():
    filtered_reviews = [review for review in row["reviews"] if review.get("review_text") != "None"]
    df.at[index, "reviews"] = filtered_reviews

                                  gmap_id  \
0   0x54cde72868493ca5:0x3bed623bec4c43e3   
1   0x54d400a3503bf3cb:0x6f54a8d273995ade   
2   0x54d4ecda5872a3d3:0x133d4cc056e83ab4   
3   0x808327b71461d6bb:0x7e1165558a98f648   
4   0x8083ff64283d3aab:0x278c940d94dac24d   
5   0x8084481ecb286ce3:0x9621a5cee38c4ec2   
6    0x80844820b85359e3:0xff414747a10c915   
7   0x8084d14c6ecc3abf:0x8516c9daf203a890   
8   0x808506ba36069ef1:0x872c89de33e28e2c   
9   0x808515c134667051:0x9a164a19ad59e8ff   
10  0x808515d26b90f18f:0x1e763dbe37ae7516   
11  0x8085613c0ba3453d:0xafafb2f005ef4730   
12  0x8085619759ff8759:0xae65b3fcdfd02e45   
13  0x808567674283bbdd:0x1b039c22a6be8de5   
14   0x808577a4077796b7:0x74c2df8373dd019   
15  0x808577b06c49fbcb:0x3c48448b009b80d3   
16  0x808579239e005685:0x85d07736600cdbc6   
17  0x80857dcdecda8877:0xc56ed45ede87f983   
18  0x808580e9b3089847:0x3a964d5d97defd44   
19  0x8085ac3b9e99ebf5:0xfcba33f78ee94975   

                                              reviews 

Justification: After reading the json file into a pandas dataframe. I found that the column and rows need to be transposed. Hence I used .T to ahieve that. Only the review contains more than 70 texts review is needed. So I used a list comprehension first to drop the review that is "None".

<div class="alert alert-block alert-success">
    
## 4.  Loading and Parsing File <a class="anchor" name="load"></a>

In this section, ....

In [39]:
print(df.shape)
def extract_reviews(df):
    df = df[df['reviews'].apply(len) >= 70].reset_index(drop=True)
    df['review_text'] = ''
    for index, row in df.iterrows():
        review_texts = []
        for review in row['reviews']:
            text = review.get('review_text', '').lower()
            # text = re.sub(r'[^\x00-\x7F]+', '', text)
            review_texts.append(text)
        df.at[index, 'review_text'] = ' '.join(review_texts)
    print(df.shape)
    return df

df_review = extract_reviews(df)

(176, 4)
(124, 5)


After dropping all the review_text which is "None", I filtered the review by whose "reviews" list has more than 70 elements by applying len() method. After filtering the reviews, I created a new dataframe to store them. After that, a for loop is used to iterate through each row of the new data frame to store the review text to a new column "review_text". And then, join the review_text list to a string for further tokenization.

<div class="alert alert-block alert-warning">
    
### 4.1. Tokenization <a class="anchor" name="tokenize"></a>

Tokenization is a principal step in text processing and producing unigrams. In this section, ....

In [40]:
#Correct version
stopwords_list = []
with open(r'Student Data/stopwords_en.txt', 'r') as file:
    for line in file:
        stopwords_list.append(line.strip())
df_review["tokennized_review"] = None
for index,row in df_review.iterrows():
    # Tokenization
    review_text = row["review_text"]
    tokenizer = RegexpTokenizer(r"[a-zA-Z]+")
    unigram_tokens = tokenizer.tokenize(review_text)

    # Remove tokens less than three
    unigram_tokens = [token for token in unigram_tokens if len(token) >= 3]

    # Remove independent stop words
    stopped_tokens = [w for w in unigram_tokens if w not in stopwords_list]
    #Stemming
    stemmer = PorterStemmer()
    stemed_tokens = [stemmer.stem(token) for token in stopped_tokens]
    df_review.at[index, "tokennized_review"] = stemed_tokens
    # print(mwe_tokens)

1. Read the stopwords txt and append them to a list called stopwords_lis
2. Iterate through the rows of df_reviews
3. Task a finished: Tokenize the review_text of each row by RegexpTokenizer
Justification: To perform further processing like remove stop words, Stemmentization etc, Words need to be tokenized first.
4. Task e finished: Used list comprehension to  filter out the tokens to be length longer than3
Justification: Words's Lenth less than 3 are always meanningless words. Remove than at the beginning can help improve efficency of data processing.
5. Task b1 finishied: Used list comprehension to filter the unigram_tokens by word not in the stop_word list.
Justification: Stop words are considered noise that needs to be removed. Remove them before stemmatization, because if the stopwords are stemmed first, the words would be transformed that not being able to match the stopword list.
6. Stemming the remaining tokens to their root forms using the Porter Stemmer.
Justification: To identify context dependent stopwords, words need to be stemmed before removing them

Explanation on result:

In [41]:
# Remove context dependent stopwords
word_in_business = {}
total_businesses = df_review.shape[0]

for index, row in df_review.iterrows():
    gmap_id = row['gmap_id']
    unique_words = set(row['tokennized_review'])
    for word in unique_words:
        if word not in word_in_business:
            word_in_business[word] = set()
        word_in_business[word].add(gmap_id)

context_dependent_stopwords = [word for word, business_set in word_in_business.items()
                                if len(business_set) / total_businesses > 0.95]

for index,row in df_review.iterrows():
    stopped_tokens = [w for w in row['tokennized_review'] if w not in context_dependent_stopwords]
    df_review.at[index, "tokennized_review"] = stopped_tokens


1.Identifying and removing context-dependent stopwords, which are words that appear in more than 95% of businesses.
Justification: For example: "The words "restaurant" and "restaurants" are treated separately. "Restaurant" might appear in 80% of reviews, and "restaurants" in 20%. Individually, neither might exceed a frequency threshold (like 95%). Thus, neither is removed as a context-dependent stopword.

Explanation on result:

In [42]:
# Remove rare words
word_in_business = {}
total_businesses = df_review.shape[0]

for index, row in df_review.iterrows():
    gmap_id = row['gmap_id']
    unique_words = set(row['tokennized_review'])
    for word in unique_words:
        if word not in word_in_business:
            word_in_business[word] = set()
        word_in_business[word].add(gmap_id)

rare_words = [word for word, business_set in word_in_business.items()
                                if len(business_set) / total_businesses < 0.05]

for index,row in df_review.iterrows():
    rared_tokens = [w for w in row['tokennized_review'] if w not in rare_words]
    df_review.at[index, "tokennized_review"] = rared_tokens


1. The purpose of this code is to remove rare words from the tokenized reviews. Rare words are defined as those that appear in less than 5% of the businesses in the dataset.
Justification: This step should be done after stemmatization. For example,"running," "ran," and "runs" all stem to "run." If rare word removal were done before stemming, each form would be treated as a separate entity, potentially missing the fact that collectively they might not be rare.

In [43]:
# Bigram words
all_tokens = sum(df_review["tokennized_review"], [])
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(all_tokens)
bigram_token = finder.nbest(bigram_measures.pmi, 200)  # Get the top 200 bigrams
tokenizer = MWETokenizer(bigram_token)
for index, row in df_review.iterrows():
    mwe_tokens = tokenizer.tokenize(row["tokennized_review"])
    df_review.at[index, "tokennized_review"] = mwe_tokens  
# print(df_review.head())

1. Used MWETokenizer with pmi to find the first 200 bigrams and iterate through each row to identify the bigram and add bigram to it.

Justification: The Pointwise Mutual Information (PMI) measure used to identify meaningful bigrams relies on the frequency and co-occurrence of words within a text. By removing unimportant or noisy words first, the PMI calculation becomes more accurate and reflective of meaningful word associations. If bigrams were generated earlier, PMI scores could be skewed by the presence of irrelevant or overly common tokens, leading to less meaningful bigrams.

In [44]:
# Export word count txt file
all_tokens_per_doc = [token for token in df_review["tokennized_review"]]

1. Sorted the cleaned tokens and write them to test_vocab.txt by desired format.

At this stage, all reviews for each PID are tokenized and are stored as a value in the new dictionary (separetely for each day).

-------------------------------------

<div class="alert alert-block alert-warning">
    
### 4.2. Whatever else <a class="anchor" name="whetev"></a>

<div class="alert alert-block alert-warning">
    
### 4.3. Generate numerical representation<a class="anchor" name="bigrams"></a>

One of the tasks is to generate the numerical representation for all tokens in abstract.  .....

In [45]:
#some code
vectorizer = CountVectorizer(analyzer="word")
documents = [' '.join(tokens) for tokens in all_tokens_per_doc]

data_features = vectorizer.fit_transform(documents)

vocab = vectorizer.get_feature_names_out()
vocab_dist = {word: index for index, word in enumerate(vocab)}
with open("test_vocab.txt",'w') as file:
    for index,word in enumerate(vocab_dist):
        file.write(f"{word}:{ index}\n")

print(vocab_dist)
output_countvec = []
for doc_index, gmap_id in enumerate(df_review["gmap_id"]):
    doc_vector = data_features[doc_index]
    token_counts = zip(doc_vector.indices, doc_vector.data)
    formatted_counts = [f"{index}:{frequency}" for index,frequency in token_counts] 
    output_line = f"{gmap_id} " + ", ".join(formatted_counts)
    output_countvec.append(output_line)

{'abil': 0, 'absolut': 1, 'accept': 2, 'access': 3, 'accessori': 4, 'accid': 5, 'accommod': 6, 'account': 7, 'accur': 8, 'acknowledg': 9, 'act': 10, 'action': 11, 'activ': 12, 'actual': 13, 'ad': 14, 'add': 15, 'addict': 16, 'addit': 17, 'address': 18, 'adequ': 19, 'adjust': 20, 'admit': 21, 'admit_purpos': 22, 'ador': 23, 'adult': 24, 'advanc': 25, 'advantag': 26, 'advertis': 27, 'advic': 28, 'advis': 29, 'afford': 30, 'afraid': 31, 'afternoon': 32, 'age': 33, 'agenc': 34, 'agent': 35, 'aggress': 36, 'ago': 37, 'agre': 38, 'ahead': 39, 'air': 40, 'allow': 41, 'alot': 42, 'alright': 43, 'altern': 44, 'amaz': 45, 'amazingli': 46, 'ambianc': 47, 'ambienc': 48, 'amd': 49, 'amd_helpful': 50, 'amen': 51, 'america': 52, 'american': 53, 'amount': 54, 'ampl': 55, 'angel': 56, 'anim': 57, 'annoy': 58, 'answer': 59, 'anticip': 60, 'anxieti': 61, 'anymor': 62, 'anytim': 63, 'apolog': 64, 'apolog_inconveni': 65, 'apologet': 66, 'app': 67, 'appar': 68, 'appear': 69, 'appet': 70, 'appetit': 71, 'app

Random descriptions and justification ...

At this stage, we have a dictionary of tokenized words, whose keys are indicative of....

-------------------------------------

#### Whatever else <a class="anchor" name="whatev1"></a>

<div class="alert alert-block alert-success">
    
## 5. Writing Output Files <a class="anchor" name="write"></a>

In [46]:
with open("countvec_test.txt","w") as file:
    for line in output_countvec:
        file.write(line + "\n")

files need to be generated:
* Vocabulary list
* Sparse matrix (count_vectors)

This is performed in the following sections.

<div class="alert alert-block alert-warning">
    
### 5.1. Vocabulary List <a class="anchor" name="write-vocab"></a>

List of vocabulary should also be written to a file, sorted alphabetically, with their reference codes in front of them. This file also refers to the sparse matrix in the next file. For this purpose, .....

In [47]:
#some code

<div class="alert alert-block alert-warning">
    
### 5.2. Sparse Matrix <a class="anchor" name="write-sparseMat"></a>

For writing sparse matrix for a paper, we firstly calculate the frequency of words for that paper ....

In [48]:
#some code

-------------------------------------

<div class="alert alert-block alert-success">
    
## 6. Summary <a class="anchor" name="summary"></a>

.....

-------------------------------------

<div class="alert alert-block alert-success">
    
## 7. References <a class="anchor" name="Ref"></a>

[1] Pandas dataframe.drop_duplicates(), https://www.geeksforgeeks.org/python-pandas-dataframe-drop_duplicates/, Accessed 13/08/2022.



## --------------------------------------------------------------------------------------------------------------------------